# Задание 1

In [14]:
import urllib.request as urllib2
from bs4 import *
from urllib.parse  import urljoin
from random import randrange


urlList = set()
urlList.add('https://www.biography.com/activist/jesse-jackson')
print(urlList)

def crawl(pages, depth=None):
    indexed_url = [] # a list for the main and sub-HTML websites in the main website
    for i in range(depth):
        for page in pages:
            if page not in indexed_url:
                indexed_url.append(page)
                try:
                    c = urllib2.urlopen(page)
                except:
                    print( "Could not open %s" % page)
                    continue
                soup = BeautifulSoup(c.read())
                links = soup('a') #finding all the sub_links
                for link in links:
                    if 'href' in dict(link.attrs):
                        url = urljoin(page, link['href'])
                        if url.find("'") != -1:
                                continue
                        url = url.split('#')[0] 
                        if url[0:4] == 'http':
                                indexed_url.append(url)
        pages = indexed_url
    indexed_url = [curr for curr in indexed_url if 'activist' in curr 
                   or 'people' in curr or 'musician' in curr or 'personality' in curr 
                   or 'political-figure' in curr or 'us-president' in curr]
    return indexed_url

while len(urlList) < 100:
    for i in list(urlList):
        urls = crawl([i], depth=1)
        urlList.update(urls)
        if(len(urlList) > 100):
            break
    print(len(urlList))
    if(len(urlList) > 150):
        break
    

# некоторые ссылки могут быть битые, поэтому заранне сохраним 500 страниц
with open('search/links.txt', 'w') as f:
    for item in urlList:
        f.write("%s\n" % item)

{'https://www.biography.com/activist/jesse-jackson'}
20
158


In [15]:
import urllib
from random import randrange
counter = 0
with open('search/index.txt', 'w') as f:
    with open('search/links.txt') as file:
        for line in file:
            name = 'search/task1/' + str(randrange(5000)) + '.html'
            try:
                urllib.request.urlretrieve(line, name)
                counter += 1
                print(counter)
                result_string = line.replace("\n", "") + ',' + name + '\n'
                f.write(result_string)
                if(counter == 100):
                    print('Time to break up')
                    f.close()
                    break
            except:
                print( "Could not open %s" % line)
                continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Time to break up


# Задание 2

In [30]:
from bs4 import BeautifulSoup

import nltk
import re
nltk.download('punkt')
nltk.download("stopwords")

from nltk import sent_tokenize
from nltk.corpus import stopwords

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

#получаем список стоп слов
stop = stopwords.words('english')

CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/simononboard/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/simononboard/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/simononboard/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [53]:
import os
arr = os.listdir('search/task1/')
counter = 0
for file in arr:
    with open('search/task1/' + file, 'r', encoding="ISO-8859-1") as fp:
        tokens = []
        dictionary = {}
        data = fp.read()
        print(file)
        pattern = r'<[ ]*script.*?\/[ ]*script[ ]*>'  # mach any char zero or more times
        data = re.sub(pattern, '  ', data, flags=(re.IGNORECASE | re.MULTILINE | re.DOTALL))
        pattern = r'<[ ]*style.*?\/[ ]*style[ ]*>'  # mach any char zero or more times
        data = re.sub(pattern, '  ', data, flags=(re.IGNORECASE | re.MULTILINE | re.DOTALL))
        pattern = r'<[ ]*meta.*?>'  # mach any char zero or more times
        data = re.sub(pattern, '  ', data, flags=(re.IGNORECASE | re.MULTILINE | re.DOTALL))
        cleantext = re.sub(CLEANR, ' ', data)
        lines = [cleantext]
        if(cleantext.find('\n') != -1):
            lines = cleantext.split('\n')
            for line in lines:
                if line:
                    sentences = sent_tokenize(line, language="english")
                    for cur in sentences:
                        cur = cur.lower()
                        cur = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", cur)
                        cur = re.sub(r"\d+", " ", cur)
                        cur = '  '.join([word for word in cur.split() if word not in (stop)])
                        words = cur.split()
                        for word in words:
                            current_lemma = WordNetLemmatizer().lemmatize(word)
                            tokens.append(word)
                            if current_lemma not in dictionary:
                                dictionary[current_lemma] = set()
                            dictionary[current_lemma].add(word)   
        with open('search/task2/' + file.replace(".html", "") + "_tokens" + '.txt', 'w') as f:
            for item in tokens:
                f.write("%s\n" % item)
        with open('search/task2/' + file.replace(".html", "") + "_dictionary" + '.txt', 'w') as f:
            for key, value in dictionary.items():
                result_string = key + ": "
                if( len(value) >= 2 ) :
                    print( key + ": " + str(value))
                for val in value:
                    result_string += val + " "
                f.write("%s\n" % result_string)    
        counter += 1

print(counter)

340.html
age: {'age', 'ages'}
family: {'families', 'family'}
crime: {'crimes', 'crime'}
u: {'us', 'u'}
term: {'term', 'terms'}
policy: {'policies', 'policy'}
photo: {'photos', 'photo'}
fact: {'facts', 'fact'}
school: {'school', 'schools'}
law: {'law', 'laws'}
place: {'place', 'places'}
year: {'years', 'year'}
view: {'view', 'views'}
american: {'americans', 'american'}
right: {'right', 'rights'}
way: {'way', 'ways'}
win: {'win', 'wins'}
lead: {'leads', 'lead'}
get: {'get', 'gets'}
republican: {'republicans', 'republican'}
job: {'job', 'jobs'}
sound: {'sounds', 'sound'}
state: {'state', 'states'}
senator: {'senators', 'senator'}
close: {'close', 'closes'}
office: {'offices', 'office'}
nation: {'nations', 'nation'}
kid: {'kids', 'kid'}
day: {'day', 'days'}
child: {'children', 'child'}
passage: {'passage', 'passages'}
window: {'windows', 'window'}
garden: {'gardens', 'garden'}
receiver: {'receivers', 'receiver'}
coach: {'coaches', 'coach'}
party: {'party', 'parties'}
member: {'member', 'me

jackson: {'jackson', 'jacksons'}
child: {'children', 'child'}
song: {'song', 'songs'}
video: {'video', 'videos'}
u: {'us', 'u'}
fact: {'facts', 'fact'}
brother: {'brother', 'brothers'}
life: {'life', 'lives'}
sibling: {'sibling', 'siblings'}
year: {'years', 'year'}
record: {'record', 'records'}
hit: {'hits', 'hit'}
chart: {'chart', 'charts'}
artist: {'artists', 'artist'}
album: {'albums', 'album'}
wife: {'wives', 'wife'}
245.html
group: {'groups', 'group'}
biography: {'biographies', 'biography'}
video: {'video', 'videos'}
u: {'us', 'u'}
american: {'americans', 'american'}
sport: {'sports', 'sport'}
253.html
clinton: {'clinton', 'clintons'}
u: {'us', 'u'}
term: {'term', 'terms'}
choice: {'choices', 'choice'}
fact: {'facts', 'fact'}
life: {'life', 'lives'}
year: {'years', 'year'}
state: {'state', 'states'}
president: {'president', 'presidents'}
woman: {'woman', 'women'}
right: {'right', 'rights'}
schedule: {'schedule', 'schedules'}
child: {'children', 'child'}
science: {'science', 'scien

age: {'age', 'ages'}
family: {'families', 'family'}
crime: {'crimes', 'crime'}
u: {'us', 'u'}
term: {'term', 'terms'}
policy: {'policies', 'policy'}
photo: {'photos', 'photo'}
fact: {'facts', 'fact'}
school: {'school', 'schools'}
law: {'law', 'laws'}
place: {'place', 'places'}
year: {'years', 'year'}
view: {'view', 'views'}
american: {'americans', 'american'}
right: {'right', 'rights'}
way: {'way', 'ways'}
win: {'win', 'wins'}
lead: {'leads', 'lead'}
get: {'get', 'gets'}
republican: {'republicans', 'republican'}
job: {'job', 'jobs'}
sound: {'sounds', 'sound'}
state: {'state', 'states'}
senator: {'senators', 'senator'}
close: {'close', 'closes'}
office: {'offices', 'office'}
nation: {'nations', 'nation'}
kid: {'kids', 'kid'}
day: {'day', 'days'}
child: {'children', 'child'}
passage: {'passage', 'passages'}
window: {'windows', 'window'}
garden: {'gardens', 'garden'}
receiver: {'receivers', 'receiver'}
coach: {'coaches', 'coach'}
party: {'party', 'parties'}
member: {'member', 'members'}
l

celebrity: {'celebrities', 'celebrity'}
culture: {'cultures', 'culture'}
u: {'us', 'u'}
term: {'term', 'terms'}
policy: {'policies', 'policy'}
choice: {'choices', 'choice'}
fact: {'facts', 'fact'}
senator: {'senators', 'senator'}
american: {'americans', 'american'}
president: {'president', 'presidents'}
state: {'state', 'states'}
law: {'law', 'laws'}
school: {'school', 'schools'}
war: {'wars', 'war'}
challenge: {'challenge', 'challenges'}
meet: {'meet', 'meets'}
thing: {'thing', 'things'}
responsibility: {'responsibilities', 'responsibility'}
right: {'right', 'rights'}
commitment: {'commitment', 'commitments'}
help: {'helps', 'help'}
opportunity: {'opportunities', 'opportunity'}
individual: {'individual', 'individuals'}
america: {'america', 'americas'}
one: {'ones', 'one'}
nation: {'nations', 'nation'}
justice: {'justices', 'justice'}
change: {'change', 'changes'}
way: {'way', 'ways'}
sister: {'sisters', 'sister'}
hope: {'hope', 'hopes'}
working: {'workings', 'working'}
guarantee: {'gu

celebrity: {'celebrities', 'celebrity'}
culture: {'cultures', 'culture'}
u: {'us', 'u'}
term: {'term', 'terms'}
policy: {'policies', 'policy'}
choice: {'choices', 'choice'}
fact: {'facts', 'fact'}
senator: {'senators', 'senator'}
american: {'americans', 'american'}
president: {'president', 'presidents'}
state: {'state', 'states'}
law: {'law', 'laws'}
school: {'school', 'schools'}
war: {'wars', 'war'}
challenge: {'challenge', 'challenges'}
meet: {'meet', 'meets'}
thing: {'thing', 'things'}
responsibility: {'responsibilities', 'responsibility'}
right: {'right', 'rights'}
commitment: {'commitment', 'commitments'}
help: {'helps', 'help'}
opportunity: {'opportunities', 'opportunity'}
individual: {'individual', 'individuals'}
america: {'america', 'americas'}
one: {'ones', 'one'}
nation: {'nations', 'nation'}
justice: {'justices', 'justice'}
change: {'change', 'changes'}
way: {'way', 'ways'}
sister: {'sisters', 'sister'}
hope: {'hope', 'hopes'}
working: {'workings', 'working'}
guarantee: {'gu

fact: {'facts', 'fact'}
u: {'us', 'u'}
term: {'term', 'terms'}
opportunity: {'opportunities', 'opportunity'}
time: {'time', 'times'}
life: {'life', 'lives'}
family: {'families', 'family'}
member: {'member', 'members'}
woman: {'woman', 'women'}
state: {'state', 'states'}
child: {'children', 'child'}
year: {'years', 'year'}
look: {'look', 'looks'}
congresswoman: {'congresswoman', 'congresswomen'}
caucus: {'caucuses', 'caucus'}
campaign: {'campaign', 'campaigns'}
attack: {'attacks', 'attack'}
tax: {'tax', 'taxes'}
affair: {'affairs', 'affair'}
518.html
right: {'right', 'rights'}
u: {'us', 'u'}
fact: {'facts', 'fact'}
american: {'americans', 'american'}
citizen: {'citizens', 'citizen'}
leader: {'leaders', 'leader'}
black: {'blacks', 'black'}
year: {'years', 'year'}
porter: {'porters', 'porter'}
position: {'position', 'positions'}
day: {'days', 'day'}
life: {'life', 'lives'}
4685.html
song: {'song', 'songs'}
fact: {'facts', 'fact'}
life: {'life', 'lives'}
like: {'likes', 'like'}
love: {'lov

jackson: {'jackson', 'jacksons'}
fact: {'facts', 'fact'}
child: {'children', 'child'}
life: {'life', 'lives'}
sibling: {'sibling', 'siblings'}
time: {'time', 'times'}
year: {'years', 'year'}
award: {'award', 'awards'}
drug: {'drug', 'drugs'}
action: {'actions', 'action'}
lawyer: {'lawyer', 'lawyers'}
artist: {'artists', 'artist'}
hit: {'hits', 'hit'}
4683.html
castro: {'castro', 'castros'}
cuban: {'cubans', 'cuban'}
brother: {'brother', 'brothers'}
term: {'term', 'terms'}
policy: {'policies', 'policy'}
fact: {'facts', 'fact'}
life: {'life', 'lives'}
president: {'president', 'presidents'}
minister: {'ministers', 'minister'}
decade: {'decades', 'decade'}
state: {'state', 'states'}
year: {'years', 'year'}
way: {'way', 'ways'}
hand: {'hands', 'hand'}
woman: {'woman', 'women'}
2604.html
right: {'right', 'rights'}
video: {'video', 'videos'}
u: {'us', 'u'}
policy: {'policies', 'policy'}
fact: {'facts', 'fact'}
state: {'state', 'states'}
american: {'americans', 'american'}
leader: {'leaders', 

# Задание 3

In [55]:
import glob

In [56]:
arr = glob.glob("search/task2/*_tokens.txt")
counter = 0
token_set = set()
tokens_dictionary = {}

result_dictionary = {}

for file in arr:
    with open(file, 'r') as fp:
        data = fp.read()
        name = file.replace("_tokens.txt", "").replace("search/task2/", "")
        data_list = data.splitlines()
        tokens_dictionary[name] = data_list
        token_set.update(data_list)


for token in token_set:
    result_dictionary[token] = []
    for key, value in tokens_dictionary.items():
        if token in value:
            result_dictionary[token].append(key)

            
with open('search/task3/inverted_index.txt', 'w') as f:
            for key, value in result_dictionary.items():
                result_string = key + " "
                for val in value:
                    result_string += val + " "
                f.write("%s\n" % result_string) 


with open('search/task3/inverted_index_2.txt', 'w') as f:
            for key, value in result_dictionary.items():
                result_string = {"count": len(value), "inverted_array": value, "word": key}
                f.write("%s\n" % str(result_string)) 

# Булевый поиск


In [83]:
import re

token_corpus_list = {}
index_page={}

with open('search/task3/inverted_index_2.txt', 'r') as f:
    data = f.readlines()
    for elem in data:
        elem = elem.replace("\'", "\"")
        string_data = json.loads(elem)
        token_corpus_list[string_data['word']] = string_data['inverted_array']
        
with open('search/index.txt', 'r') as f:
    data = f.readlines()
    for elem in data:
        res = re.split(',', elem)
        index_page[res[1].replace("\n", "")] = res[0]
        
requests = ['pool & renowned & respects','crow & newtown']



for i in requests:
    result = set()
    links = []
    res = re.split('&', i)
    for j in res:
        j = j.strip()
#         print(token_corpus_list[j])
        if(len(result) == 0):
            result.update(token_corpus_list[j])
        else:
            result = result.intersection(token_corpus_list[j])
#     print(result)
    for k in result:
        links.append(index_page['search/task1/' + k + '.html'])
    print("___________________________________________________")
    print("Request: " + i)
    print("Result: " + str(links))
    print("Result size: " + str(len(links)))
    




___________________________________________________
Request: pool & renowned & respects
Result: []
Result size: 0
___________________________________________________
Request: crow & newtown
Result: ['https://www.biography.com/us-president/barack-obama', 'https://www.biography.com/people/barack-obama-12782369']
Result size: 2


# Задание 4

In [57]:
import collections
import json
import math

In [58]:
arr = glob.glob("search/task2/*_tokens.txt")
token_corpus_count = {}

tokens_dictionary = {}

def compute_tf(text):
    tf_text = collections.Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text

def compute_idf(word, corpus):
    return math.log10(len(tokens_dictionary)/token_corpus_count[word])


with open('search/task3/inverted_index_2.txt', 'r') as f:
    data = f.readlines()
    for elem in data:
        elem = elem.replace("\'", "\"")
        string_data = json.loads(elem)
        token_corpus_count[string_data['word']] = string_data['count']
        
for file in arr:
    with open(file, 'r') as fp:
        data = fp.read()
        data_list = data.splitlines()
        name = file.replace("_tokens.txt", "").replace("search/task2/", "")
        tokens_dictionary[name] = data_list

print(len(tokens_dictionary))

for key, value in tokens_dictionary.items():
    tf_idf_dictionary = {}
    computed_tf = compute_tf(value)
    for word in computed_tf:
        idf = compute_idf(word,token_corpus_count)
        tf_idf_dictionary[word] = [idf, computed_tf[word] * idf]
    with open('search/task4/' + key + '.txt', 'w') as f:
            for key1, value1 in tf_idf_dictionary.items():
                result_string = key1 + " " + str(value1[0]) + " " + str(value1[1])
                f.write("%s\n" % result_string)    


100


In [60]:
arr = glob.glob("search/task2/*_tokens.txt")
token_corpus_count = {}

tokens_dictionary = {}

def compute_tf(text):
    tf_text = collections.Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text

def compute_idf(word, corpus):
    return math.log10(len(tokens_dictionary)/token_corpus_count[word])


with open('search/task3/inverted_index_2.txt', 'r') as f:
    data = f.readlines()
    for elem in data:
        elem = elem.replace("\'", "\"")
        string_data = json.loads(elem)
        token_corpus_count[string_data['word']] = string_data['count']
        
for file in arr:
    with open(file, 'r') as fp:
        data = fp.read()
        data_list = data.splitlines()
        name = file.replace("_tokens.txt", "").replace("search/task2/", "")
        tokens_dictionary[name] = data_list

lemma_dictionary = {}
arr = glob.glob("search/task2/*_dictionary.txt")

for file in arr:
    with open(file, 'r') as fp:
        data = fp.read()
        data_list = data.splitlines()
        name = file.replace("_dictionary.txt", "").replace("search/task2/", "")
        lemma_dictionary[name] = data_list
        
for key, value in tokens_dictionary.items():
    tf_idf_dictionary = {}
    computed_tf = compute_tf(value)
    curr_lemmas = lemma_dictionary[key]
    for lemma in curr_lemmas:
        values = lemma.split(' ')
        tf_lemma_sum = float(0)
        idf_sum = float(0)
        for i in range(1,len(values) - 1):
            tf_lemma_sum += computed_tf[values[i]]
            idf_sum += compute_idf(values[i],token_corpus_count)
        tf_idf_dictionary[values[0].replace(":","")] = [tf_lemma_sum, idf_sum, tf_lemma_sum * idf_sum]
    with open('search/task4_1/' + key + '_lemma.txt', 'w') as f:
            for key1, value1 in tf_idf_dictionary.items():
                result_string = key1 + " " + str(value1[0]) + " " + str(value1[1]) + " " + str(value1[2])
                f.write("%s\n" % result_string) 